In [192]:
import boto3
import sagemaker
from sagemaker.amazon.amazon_estimator import image_uris

session = sagemaker.Session()
bucket = session.default_bucket()

%env bucket s3://esi4testservicesaws

env: bucket=s3://esi4testservicesaws


In [193]:
region_name = boto3.Session().region_name
algorithm = sagemaker.image_uris.retrieve(region='eu-west-3', framework = "image-classification", version ="1")

print("Using algorithm %s" % algorithm)

Using algorithm 749696950732.dkr.ecr.eu-west-3.amazonaws.com/image-classification:1


In [57]:
%%sh
wget http://data.mxnet.io/data/caltech-256/caltech-256-60-train.rec
wget http://data.mxnet.io/data/caltech-256/caltech-256-60-val.rec

--2021-04-29 13:06:49--  http://data.mxnet.io/data/caltech-256/caltech-256-60-train.rec
Resolving data.mxnet.io (data.mxnet.io)... 142.250.179.115, 2a00:1450:4007:80d::2013
Connecting to data.mxnet.io (data.mxnet.io)|142.250.179.115|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://data.mxnet.io.s3-website-us-west-1.amazonaws.com/data/caltech-256/caltech-256-60-train.rec [following]
--2021-04-29 13:06:49--  http://data.mxnet.io.s3-website-us-west-1.amazonaws.com/data/caltech-256/caltech-256-60-train.rec
Resolving data.mxnet.io.s3-website-us-west-1.amazonaws.com (data.mxnet.io.s3-website-us-west-1.amazonaws.com)... 52.219.116.107
Connecting to data.mxnet.io.s3-website-us-west-1.amazonaws.com (data.mxnet.io.s3-website-us-west-1.amazonaws.com)|52.219.116.107|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 363622688 (347M) [binary/octet-stream]
Saving to: ‘caltech-256-60-train.rec.2’

     0K .......... .......... ....

In [194]:
session.upload_data(path='caltech-256-60-train.rec', bucket=bucket, key_prefix='train')
session.upload_data(path='caltech-256-60-val.rec',   bucket=bucket, key_prefix='validation')

s3_train      = 's3://{}/train/'.format(bucket)
s3_validation = 's3://{}/validation/'.format(bucket)
s3_output     = 's3://{}/output'.format(bucket)

%env s3_train      $s3_train
%env s3_validation $s3_validation
%env s3_output     $s3_output

env: s3_train=s3://sagemaker-eu-west-3-096478844572/train/
env: s3_validation=s3://sagemaker-eu-west-3-096478844572/validation/
env: s3_output=s3://sagemaker-eu-west-3-096478844572/output


In [195]:
%%sh
aws s3 ls $s3_train
aws s3 ls $s3_validation

2021-04-29 13:50:08  363622688 caltech-256-60-train.rec
2021-04-29 13:50:11  360242580 caltech-256-60-val.rec


In [233]:

ic = sagemaker.estimator.Estimator(
                                   algorithm,
                                   'arn:aws:iam::096478844572:role/awssagemaker',
                                   instance_count=1, 
                                   instance_type='ml.m5.large',
                                   output_path=s3_output,
                                   sagemaker_session=session)


In [234]:
ic.set_hyperparameters(num_layers=18,               # Train a Resnet-18 model
                       use_pretrained_model=1,      # Fine-tune on our dataset
                       image_shape="3,224,224",   # 3 channels (RGB), 224x224 pixels
                       num_classes=257,             # 256 classes + 1 clutter class
                       num_training_samples=15420,  # Number of training samples
                       mini_batch_size=128,
                       epochs=10,                  # Learn the training samples 10 times
                       learning_rate=0.01,
                       augmentation_type='crop_color_transform' # Add altered images
                       #precision_dtype='float16'    # Train at half-precision to save memory
                      )

In [235]:
train_data = sagemaker.inputs.TrainingInput(s3_train, 
                                        distribution='FullyReplicated', 
                                        content_type='application/x-recordio',
                                        s3_data_type='S3Prefix')

validation_data = sagemaker.inputs.TrainingInput(s3_validation,
                                             distribution='FullyReplicated', 
                                             content_type='application/x-recordio', 
                                             s3_data_type='S3Prefix')

data_channels = {'train': train_data, 'validation': validation_data}

In [236]:
ic.fit(inputs=data_channels, logs=True)

ClientError: An error occurred (ValidationException) when calling the CreateTrainingJob operation: Instance type ml.m5.large is not supported by algorithm image-classification; only GPU instances are supported.

In [88]:
ic_predictor = ic.deploy(initial_instance_count=1,
                         instance_type='ml.c5.18xlarge')

ValueError: Estimator is not associated with a training job

In [11]:

!wget -O /tmp/test.jpg http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/022.buddha-101/022_0019.jpg
file_name = '/tmp/test.jpg'
# test image
from IPython.display import Image
Image(file_name)

--2021-04-29 12:09:32--  http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/022.buddha-101/022_0019.jpg
Resolving www.vision.caltech.edu (www.vision.caltech.edu)... 34.208.54.77
Connecting to www.vision.caltech.edu (www.vision.caltech.edu)|34.208.54.77|:80... connected.
HTTP request sent, awaiting response... 404 Not Found
2021-04-29 12:09:33 ERROR 404: Not Found.



In [12]:
import json
import numpy as np

# Load test image from file
with open(file_name, 'rb') as f:
    payload = f.read()
    payload = bytearray(payload)

# Set content type
ic_predictor.content_type = 'application/x-image'

# Predict image and print JSON predicton
result = json.loads(ic_predictor.predict(payload))

# Print top class
index = np.argmax(result)
print("Result: label - " + object_categories[index] + ", probability - " + str(result[index]))

NameError: name 'ic_predictor' is not defined

In [13]:
ic_predictor.delete_endpoint()

NameError: name 'ic_predictor' is not defined